In [9]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn

In [10]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder

In [11]:
with open('intents.json') as file:
    data = json.load(file)

In [12]:
training_sentences =[]
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes = len(labels)

In [13]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels =lbl_encoder.transform(training_labels)

In [14]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [16]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs= epochs)

Epoch 1/500


2023-01-29 14:49:39.647349: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 0s 5ms/step - loss: 2.0798 - accuracy: 0.0909
Epoch 2/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0789 - accuracy: 0.0909
Epoch 3/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0785 - accuracy: 0.0909
Epoch 4/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0782 - accuracy: 0.0909
Epoch 5/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0779 - accuracy: 0.0909
Epoch 6/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0776 - accuracy: 0.0909
Epoch 7/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0774 - accuracy: 0.0909
Epoch 8/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0772 - accuracy: 0.0909
Epoch 9/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0769 - accuracy: 0.0909
Epoch 10/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0767 - accuracy: 0.0909
Epoch 11/500
2/2 [=======================

In [18]:
model.save("chat_model")

INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets
